In [1]:
import argparse
from os.path import join
import nibabel as nib
from sklearn.model_selection import LeaveOneGroupOut
import nilearn.decoding
import numpy as np
from dataset import GetData
from permute_labels import PermLabels

# Set directories

In [2]:
DATADIR = "/home/emanuele/Documents/bnb_retreat/data/sub-01/"
OUTPUT = "searchlight_output"
MASK = "/home/emanuele/Documents/bnb_retreat/data/masks/binarized_ofc.nii.gz"

# Set relevant variables 

#### TASKS : must be a list of strings

#### RADIUS : in Nilearn the radius is in millimeters, in our case a single voxel is 2.5 millimiter and we want a 2 voxel radii (5).

In [3]:
TASKS = ["col", "pl"]
RADIUS = 5
SUBJ = "01"
CLF = "svc" # support vector classifier

# Get the data using GetData

In [4]:
dataset = GetData(
    tasks=TASKS, labels={"_f_": 1, "_nf_": 2}, group_filter=TASKS
)
data = dataset(DATADIR)
print(data)

{'data': array(['/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_f_col_block_3_run-3_zmap.nii.gz',
       '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_f_col_block_1_run-2_zmap.nii.gz',
       '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_nf_col_block_1_run-2_zmap.nii.gz',
       '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_nf_col_block_1_run-1_zmap.nii.gz',
       '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_f_col_block_2_run-2_zmap.nii.gz',
       '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_f_col_block_1_run-1_zmap.nii.gz',
       '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_nf_col_block_2_run-2_zmap.nii.gz',
       '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_f_col_block_2_run-1_zmap.nii.gz',
       '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_f_col_block_1_run-3_zmap.nii.gz',
       '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_nf_col_block_3_run-3_zmap.nii.gz',
       

# Now set the cross-validation of choice

In [5]:
CV = LeaveOneGroupOut()

### Let's check what kind of thing a cross validator returns:

In [6]:
for fold, (train, test) in enumerate(CV.split(data["data"], data["labels"], data["groups"])):
    print(f"fold number: {fold+1}")
    print("Training data:")
    print(data["data"][train])
    print("Test data")
    print(data["data"][test])

fold number: 1
Training data:
['/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_f_pl_block_1_run-1_zmap.nii.gz'
 '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_nf_pl_block_3_run-1_zmap.nii.gz'
 '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_nf_pl_block_2_run-1_zmap.nii.gz'
 '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_f_pl_block_1_run-3_zmap.nii.gz'
 '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_nf_pl_block_3_run-2_zmap.nii.gz'
 '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_f_pl_block_2_run-2_zmap.nii.gz'
 '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_f_pl_block_3_run-1_zmap.nii.gz'
 '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_f_pl_block_3_run-3_zmap.nii.gz'
 '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_f_pl_block_1_run-2_zmap.nii.gz'
 '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_f_pl_block_2_run-3_zmap.nii.gz'
 '/home/emanuele/Documents/bnb_retreat/data/sub-01/sub-01_nf_pl_blo

# Instantiate the Searchlight class

In [7]:
SL = nilearn.decoding.SearchLight(
        mask_img=MASK,
        # process_mask_img=process_mask_img,
        radius=RADIUS,
        estimator=CLF,
        n_jobs=-1,
        verbose=1,
        cv=CV,
        scoring="accuracy",
    )

# Now fit the searchlight and get the scores

In [8]:
SL.fit(data["data"], data["labels"], data["groups"])
scores = SL.scores_

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed:    2.9s remaining:    8.8s
Job #6, processed 0/409 voxels (0.00%, 46 seconds remaining)
/home/emanuele/Documents/bnb_retreat/env/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
Job #6, processed 10/409 voxels (2.44%, 1 seconds remaining)
Job #6, processed 20/409 voxels (4.89%, 1 seconds remaining)
Job #6, processed 30/409 voxels (7.33%, 1 seconds remaining)
Job #6, processed 40/409 voxels (9.78%, 1 seconds remaining)
Job #6, processed 50/409 voxels (12.22%, 1 seconds remaining)
Job #6, processed 60/409 voxels (14.67%, 1 seconds remaining)
Job #6, processed 70/409 voxels (17.11%, 1 seconds remaining)
/home/emanuele/Documents/bnb_retreat/env/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase

# Let's make a function that saves the results as nifti files.

In [9]:
def np2nii(img, scores, filename):
    """
    It saves data into a nifti file
    by leveraging on another image
    of the same size.
    Parameters
    ----------
    img : nifti file (e.g a mask)
    scores : numpy array containing decoding scores
    filename : string name of the new nifti file
    Returns
    -------
    nii_file : Nifti1Image
    """
    img = nib.load(img)
    header = nib.Nifti1Header()
    affine = img.affine
    nii_file = nib.Nifti1Image(scores, affine, header)
    nib.save(nii_file, filename)
    return nii_file


# Let's save the output and print the scores

Results are saved both as **nifti** and **.npy** (a numpy format to save arrays) files.

In [10]:
output = join(
    f"{OUTPUT}",
    (
        f"sub-{SUBJ}_{TASKS[0]}_"
        f"{TASKS[1]}_radius_{int(RADIUS)}"
    ),
)
print(scores[scores > 0])
np.save(f"{output}.npy", scores)
np2nii(MASK, scores, f"{output}.nii.gz")

[0.55555556 0.55555556 0.61111111 ... 0.44444444 0.41666667 0.44444444]


# And again plot the outcome

# Permutations (getting chance maps)

We need to produce chance maps for the statistical analysis. Chance maps are simply obtained by permuting the labels in our data and running the exact same classification as we have done with the original data. Usually two kind of schemes are for label permutation:

- Only training labels get permuted and test labels are let untouched.
- Both training and test labels are permuted.

With nilearn permuting both training and test labels is quite straightforward, but we would like to permute only the
training data. Hence we do the following:

# Make sure to split the cross validation
We cannot simply provide a cross validation to the searchlight class as we did above, otherwise each classification per fold would be done internally and we would not be able to split them and change the orginal labels with the permuted labels.

In [11]:
CV = LeaveOneGroupOut()

In [12]:
CV = list(CV.split(data["data"], data["labels"], data["groups"]))

# Now let's call the class that allows us to permute the labels

In [13]:
PERMUTATION = PermLabels(data["labels"], CV, data["groups"])

## let's us check what we get from the PermLabels class:

In [14]:
for i in PERMUTATION():
    print(data["labels"])
    print(i)


[ 7  1 14 13  3  4  9  6 15 16  5  0 12  8 11  2 10 17]
[ 8 15  2 17  4 12  6 14  3  7  9 16  5 13  0 11 10  1]
[14  6  7  8  2 12  5  0 16 15 13 10  3  1  9 11  4 17]
[13 11  8 16  5  7 15 10  4  0  2  1 14  6  9 17  3 12]
[ 1  3 10 14 13  9 15  2 17  8  4 11 16  0  6  5  7 12]
[ 1  3  6  9 17 11 12  2 13 16 10 15  5 14  4  7  8  0]
[ 5 13 11 17  4 10  3  9 16  7 14  2  0  1  6 12 15  8]
[ 2  0  1 12  4  8  9 13  3  6 10  5 16 17 15  7 11 14]
[10 15  8 11  6  9  7 12 14 16  4  5  3 17  1  0 13  2]
[ 5 11 17  8  0 15 12  3 13  7  2  6 14  4 16 10  1  9]
[ 9 11  5 12  6 10  1 15  2  8  4  3  0 16 17  7 13 14]
[ 7  8 16  5 15 14 11  0  4 17  1 12  6 10  2  9  3 13]
[ 2  3  6  0 14 12  1  8 17 15 13  4 16  9  7  5 11 10]
[14 11  5 12 13 10  9  3 16  6  1 17  2  4 15  8  0  7]
[16 15  0  5  4  2  1  7 13 12 10  6 17  8  3 14  9 11]
[ 9  1  0  4  6  3 17 11  8 12 14 15 16 10  2 13  7  5]
[ 6  4  9  1 14 11  7 13 16 10 17 15  8 12  0  2  5  3]
[ 3  8 11 16  0 15 10  2  9  5  1 13  4 14 12  7

# Now we can run a single searchlight with permuted labels only for the training set

In [15]:
for fold, ((train, test), perm) in enumerate(zip(CV, PERMUTATION())):
    print(f"fold number: {fold}")
    print(f"train indices: {train} - test indices: {test}")
    print(
        (
            f"non-permuted train labels: {data['labels'][train]}"
            f" permuted train labels: {perm}"
        )
    )

    labels = np.copy(data["labels"]) # make a copy to prevent changing the original labels
    print(data["labels"])
    labels[train] = perm
    print(labels)

    SL = nilearn.decoding.SearchLight(
        mask_img=MASK,
        radius=RADIUS,
        estimator=CLF,
        n_jobs=-1,
        verbose=1,
        cv=[(train, test)],
        scoring="accuracy",
    )
    SL.fit(data["data"], labels, data["groups"])
    scores = SL.scores_
    output = join(
        f"{OUTPUT}",
        (
            f"sub-{SUBJ}_{TASKS[0]}_{TASKS[1]}_"
            f"radius_{int(RADIUS)}_perm_"
            f"{str(1)}_fold_{fold+1}"
        ),
    )
    np.save(f"{output}.npy", scores)

[ 4  9  3 11  5 16  8  6  1 13 15  0  2 12 14 17  7 10]
[14  9  7  0 10  1  3  5 15 17  4 16 11  2  6  8 13 12]
[13 10 16  0 12  6 15  1  2  4 11  8 14  3  9 17  5  7]
[ 2  3  6 14  4 13  8  5 11 15  9 10  7  0 12  1 17 16]
[ 9  3  6 17  2 16  7 13 15 11 12 14  8  4  5  1 10  0]
[15  1  3  7 10 14 11  6 13  9  4  8  5  2 12 17 16  0]
[15  6  1  0 10 12 14  7 11 16  4  9  3 13  8 17  2  5]
[ 9 17  8  1  0  4  7 12 11 10 13 14 16  5  6  2 15  3]
[ 6  5  0  3  1 16 13 12 11 14 10  9  7 15  4 17  8  2]
[17  4 11  9  7  0  2 13 16  1  6 12 14  3 10  8 15  5]
[16  2 13  6  5  1  4 17 11  0  3  8 10  9  7 12 15 14]
[ 7 13  3 16 17 10  2  0  1  5  8  6 15  4 11  9 12 14]
[15 12  3  9 13 16  2 14  6  0  7  1  4 10  8 11  5 17]
[10 12 15 13  5  8  1  4  0  2  3 17  6 11  7  9 16 14]
[ 5 17  4 13 15 10 14 11  0  9  1 12  2  7  6  3 16  8]
[ 2  9  1  6 16  5  4 17  8  7  0 15 12 10 14 11  3 13]
[ 7  6  4 16  0  3 17  2  1 14 13 10  5  8 15 12 11  9]
[14  0  9 16 10  4  2 12  6 17  7 11 15  5 13  3

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Job #3, processed 0/410 voxels (0.00%, 59 seconds remaining)
Job #3, processed 10/410 voxels (2.44%, 2 seconds remaining)
/home/emanuele/Documents/bnb_retreat/env/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
Job #3, processed 20/410 voxels (4.88%, 1 seconds remaining)
Job #3, processed 30/410 voxels (7.32%, 1 seconds remaining)
/home/emanuele/Documents/bnb_retreat/env/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
Job #3, processed 40/410 voxels (9.76%, 1 seconds remaining)
Job #3, processed 50/410 voxels (12.20%, 1 seconds remaining)
/home/emanuele/Documents/bnb_retreat/env/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the nu

Job #7, processed 0/409 voxels (0.00%, 90 seconds remaining)
Job #7, processed 10/409 voxels (2.44%, 1 seconds remaining)
Job #7, processed 20/409 voxels (4.89%, 1 seconds remaining)
/home/emanuele/Documents/bnb_retreat/env/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/emanuele/Documents/bnb_retreat/env/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/emanuele/Documents/bnb_retreat/env/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
Job #7, processed 30/409 voxels (7.33%, 1 seconds remaining)
/home/emanuele/Documents/bnb_retreat/env/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the num

[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    0.9s finished


fold number: 1
train indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17] - test indices: [18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35]
non-permuted train labels: [1 1 2 2 1 1 2 1 1 2 2 2 2 1 2 2 1 1] permuted train labels: [2, 2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2]
[1 1 2 2 1 1 2 1 1 2 2 2 2 1 2 2 1 1 1 2 2 1 2 1 1 1 1 1 2 2 1 2 1 2 2 2]
[2 2 1 1 2 2 1 2 1 1 1 1 1 2 1 2 2 2 1 2 2 1 2 1 1 1 1 1 2 2 1 2 1 2 2 2]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Job #8, processed 0/409 voxels (0.00%, 45 seconds remaining)
Job #8, processed 10/409 voxels (2.44%, 1 seconds remaining)
Job #8, processed 20/409 voxels (4.89%, 1 seconds remaining)
Job #8, processed 30/409 voxels (7.33%, 1 seconds remaining)
Job #8, processed 40/409 voxels (9.78%, 1 seconds remaining)
Job #8, processed 50/409 voxels (12.22%, 1 seconds remaining)
Job #8, processed 60/409 voxels (14.67%, 1 seconds remaining)
Job #8, processed 70/409 voxels (17.11%, 1 seconds remaining)
Job #8, processed 80/409 voxels (19.56%, 1 seconds remaining)
Job #8, processed 90/409 voxels (22.00%, 1 seconds remaining)
Job #8, processed 100/409 voxels (24.45%, 1 seconds remaining)
Job #8, processed 110/409 voxels (26.89%, 1 seconds remaining)
Job #8, processed 120/409 voxels (29.34%, 1 seconds remaining)
Job #8, processed 130/409 voxels (31.78%, 0 seconds remaining)
Job #8, processed 140/409 voxels (34.23%, 0 seconds remai

[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed:    0.8s remaining:    2.5s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    0.9s finished
